# <p style="text-align: center;"> <span style="color:yellowgreen"> Lane detector </span></p>

### Code structure

You already have most of the code needed to complete this assignment from Exs 4 and 5.   
It is recommended to combine these nodes into one node with only one callback.   
Usually, we prefer multiple smaller nodes for modularity purposes.  
However, this comes with the cost of sending and receiving several images which can produce lagging in embedded systems such as the Jetson Nano.   
Therefore, we recommended to combine your nodes from Exs 4 and 5 into one node that subscribe to the image from the Duckiebot camera and outputs the resulting line segments post-line detection.   
The MM publishes the camera image in the topic `/ee483mm<number>/camera_node/image/compressed`.

### Compressed Images

As we seen in notebook 02b, you need to subscribe to the output of the MM's camera.  
The camera published a CompressedImage, so we need to modify the conversion function in our OpenCV bridge:

`self.bridge.compressed_imgmsg_to_cv2(compressed_image)`  

There is no need to publish compressed images.  
**Your publishers can publish the Image message type**

### Processing type
Although the Jetson Nano is a relatively powerful embedded system, your image processing code will probably take a relatively long time to run on the robot.   
The MM camera images are published about every 50 ms.
However, your code could take a whole second to complete.   
Left to default settings, ROS will patiently compile messages in a queue waiting for you to get to them, but this will mean your images will be seconds (or more) out of date by the time your code is ready.   
To prevent this, we need to give two more parameters to the subscriber function: `queue_size` and `buff_size`
Your subscriber line should look something like this:

`rospy.Subscriber("mm_camera_topic_name", CompressedImage, self.flipper_cb, queue_size=1, buff_size=10000000)`


## Testing code

Similar to Lab 1 after uploading your Docker image to MM, we need to run the following command to update any changes you make in your lab2 package. 

**Assumptions:**
- You have created the lab2 package in notebook 02b
- You are running the command inside `lab-devel`
- The laptop is connected the `robot-lan` network
- The MM is on


In [ ]:
dts devel run -H ee483mm[number].local -s -M --cmd bash -f 

Once the MM shell appear, built and activate the catkin workspace  
Sometimes, you don't need to build or source it everytime you run dts devel run (give a try)  
It might be that the autocomplete might not work though but the rosrun still works with your package when you run it

In [ ]:
catkin_build
source /code/catkin_ws/devel/setup.bash

To exit the MM Devel container shell use command: `exit`

### Common errors
1. If you get a docker conflict saying container is already running, there are two options 

	- Attach to the running container.

		`dts devel run attach`

	- Delete the running container. Do this if you have updates 

		- Connect to shell: ssh duckie@\<MMname\>.local 

		- List docker images: docker ps 

		- Delete the container: docker rm -f \<container_ID\> 

		- Exit the shell: exit 



### Observing the camera
1. In a hostOS terminal run the following commands
	- After starting the start_gui_tools, you can also run the `rqt_graph` instead of the `rqt_image_view`.
	- More details in https://docs.duckietown.com/daffy/opmanual-duckiebot/operations/make_it_see/index.html

In [ ]:
dts start_gui_tools ee483mm[number]
rqt_image_view